# Tutorial: Continuum Tortuosity
In this tutorial we demonstrate how to compute the continuum tortuosity factors of a material based on its microstructure and constituent properties. In this example, we compute the continuum tortuosity of FiberForm, a carbon fiber based material. 

Note: The rarified tortuosity factors are not available in pumapy, but are available in the c++ version of PuMA V3.0

Note: The sample size used in this example is very small, well below the size neede for a representative volume of the sample. 

In [ ]:
# ONLY FOR GOOGLE COLAB: Run this cell only the first time you open a tutorial
if 'google.colab' in str(get_ipython()):
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !conda install -c fsemerar puma

In [1]:
import numpy as np
import os
import sys
import pumapy as puma

Next we will show a continuum tortuosity simulation based on a non-segmented representation of the material. In the example material used, the void phase is contained in grayscale range [0,89] and the solid phase is contained in the grayscale range of [90,255]. This range varies for each tomography sample.

The outputs of the continuum tortuosity simulation are the continuum tortuosity factors, the effective diffusivity, the porosity, and the steady state concentration profile

In [ ]:
# Import an example tomography file of size 200^3 and voxel length 1.3e-6
ws_fiberform = puma.import_3Dtiff(puma.path_to_example_file("200_fiberform.tif"), 1.3e-6)

# The tortuosity calculation needs to be run for each of the three simulation directions. 
# For each simulation, a concentration gradient is forced in the simulation direction, and converged to steady state

# Simulation inputs: 
#.  1. workspace - the computational domain for the simulation, containing your material microstructure
#.  2. cutoff - the grayscale values for the void phase. [0,89] for this tomography sample
#.  3. direction - the simulation direction, 'x', 'y', or 'z'
#.  4. side_bc - boundary condition in the non-simulation direction. Can be 'p' - periodic, 's' - symmetric, 'd' - dirichlet
#.  5. tolerance - accuracy of the numerical solver, defaults to 1e-4. 
#.  6. maxiter - maximum number of iterations, defaults to 10,000
#.  7. solver_type - the iterative solver used. Can be 'bicgstab', 'cg', 'gmres', or 'direct'. Defaults to 'bicgstab'

n_eff_x, Deff_x, poro, C_x = puma.compute_continuum_tortuosity(ws_fiberform, (0,89), 'x', side_bc='s', tolerance=1e-3, solver_type='cg')
n_eff_y, Deff_y, poro, C_y = puma.compute_continuum_tortuosity(ws_fiberform, (0,89), 'y', side_bc='s', tolerance=1e-3, solver_type='cg')
n_eff_z, Deff_z, poro, C_z = puma.compute_continuum_tortuosity(ws_fiberform, (0,89), 'z', side_bc='s', tolerance=1e-3, solver_type='cg')

print("Effective tortuosity factors:")
print(n_eff_x)
print(n_eff_y)
print(n_eff_z)

print("Porosity of the material:", poro)

Importing /Users/fsemerar/Documents/PuMA_playground/puma-dev/python/pumapy/data/200_fiberform.tif ... Done
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to sep up A matrix:  4.974533780999991
Solving Ax=b system ... Conjugate Gradient:
Iteration 688  Residual = 0.0019952259590620444  ... Done
Time to solve:  127.76841717299976
Computing flux from converged temperature field ... Done
Computing effective conductivity... Time to compute fluxes:  0.34023263000017323
Creating conductivity matrix ... Done
Initializing temperature field ... Done
Setting up b matrix ... Done
Assembling A matrix ... 100.0% Done
Setting up preconditioner ...Done
Time to sep up A matrix:  2.9754174359995886
Solving Ax=b system ... Conjugate Gradient:
Iteration 544  Residual = 0.0029363301244711634 

In [ ]:
# Visualizing the Concentration field: 
puma.render_volume(C_x, solid_color=None, notebook=True, cmap='jet')

Below is an example of the exact same continuum tortuosity simulation, but now performed on a segmented image. If done correctly, both should produce identical results. 

In [ ]:
# Segments the image. All values >= 90 are set to 1, and all values <90 are set to 0
ws_fiberform.binarize(90)

# Simulation inputs: 
#.  1. workspace - the computational domain for the simulation, containing your material microstructure
#.  2. cutoff - the grayscale values for the void phase. [0,89] for this tomography sample
#.  3. direction - the simulation direction, 'x', 'y', or 'z'
#.  4. side_bc - boundary condition in the non-simulation direction. Can be 'p' - periodic, 's' - symmetric, 'd' - dirichlet
#.  5. tolerance - accuracy of the numerical solver, defaults to 1e-4. 
#.  6. maxiter - maximum number of iterations, defaults to 10,000
#.  7. solver_type - the iterative solver used. Can be 'bicgstab', 'cg', 'gmres', or 'direct'. Defaults to 'bicgstab'

n_eff_x, Deff_x, poro, C_x = puma.compute_continuum_tortuosity(ws_fiberform, (0,0), 'x', side_bc='s', tolerance=1e-3, solver_type='cg')
n_eff_y, Deff_y, poro, C_y = puma.compute_continuum_tortuosity(ws_fiberform, (0,0), 'y', side_bc='s', tolerance=1e-3, solver_type='cg')
n_eff_z, Deff_z, poro, C_z = puma.compute_continuum_tortuosity(ws_fiberform, (0,0), 'z', side_bc='s', tolerance=1e-3, solver_type='cg')

print("Effective tortuosity factors:")
print(n_eff_x)
print(n_eff_y)
print(n_eff_z)

print("Porosity of the material:", poro)

In [ ]:
# Visualizing the Concentration field: 
puma.render_volume(C_x, solid_color=None, notebook=True, cmap='jet')